## GeoCat experimental

In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)

CITY="lasvegas"


In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",nrows=9900,converters={'categories':string_to_array})
users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
46TNf-5T5EcNFRJPYv_nrw,ujmEBvifdJM6h6RLv4wQIg,2015-08-19 16:39:53,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
56iEnLi8jR--2ranjPSQ4w,ujmEBvifdJM6h6RLv4wQIg,2013-08-04 10:55:04,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
UDKqdVT0FrpL19shSOqgow,ujmEBvifdJM6h6RLv4wQIg,2017-03-15 23:27:38,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
Nw6Jza_cb0aqd4PT769Vzw,ujmEBvifdJM6h6RLv4wQIg,2013-01-04 22:04:02,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
3b5EsiTkqgdbxPXz7LfBpA,ujmEBvifdJM6h6RLv4wQIg,2014-08-11 23:57:54,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0


In [3]:
df_poi=pd.read_csv("../data/poi/"+CITY+".csv",converters={'categories':string_to_array})

In [4]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [5]:
users.count()

3817

### load poi neighbor

In [6]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [7]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)

for user_id in users:
    tmp_rec_list=rec.mostpopular(df_checkin.reset_index(),N)
    tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
    #print(tmp_rec_list)
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi,on='business_id').drop_duplicates(subset=df_poi.columns.difference(['categories'])).reset_index(drop=True)
    #print(tmp_rec_list)
    
    # User visits
    df_user_checkin=df_checkin.loc[[user_id]].reset_index(drop=True)

    business_cover=dict()
    for business_id in df_user_checkin['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.75 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
    relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)
    

    # post-processing for diversification
    for i in range(K):
        start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            poi_neighbors=df_poi_neighbor.loc[poi_candidate.business_id].neighbors
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories_user,rec_list_categories,poi_neighbors)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate
        #print(poi_to_insert)    
        stop = timeit.default_timer()
        print('Time:', stop - start)
        if poi_to_insert is not None:
            poi_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
    
    print(user_id)        
    print(rec_list[['business_id','score']])
    break
    
    if i % 1111 ==10:
        break



Time: 0.039082228002371266
Time: 0.11766132800039486
Time: 0.14033512300011353
Time: 0.18016907700075535
Time: 0.21566409799925168
Time: 0.24300938599844812
Time: 0.276902980000159
Time: 0.30146535099993343
Time: 0.32243477199881454
Time: 0.33728861900090124
Time: 0.36846766600137926
Time: 0.37409307900088606
Time: 0.3886585429972911
Time: 0.40869908799868426
Time: 0.426983145000122
Time: 0.4439976100002241
Time: 0.46200554199822363
Time: 0.48047671300082584
Time: 0.5097491919987078
Time: 0.5416192949996912
46TNf-5T5EcNFRJPYv_nrw
               business_id     score
25  ujmEBvifdJM6h6RLv4wQIg  0.499464
0   IMLrj2klosTFvPRLv56cng  0.053183
0   XXW_OFaYQkkGOGniujZFHg  0.046226
1   WRQ-9LluyivReFiQZFUujw  0.042844
1   Vs7gc9EE3k9wARuUcN9piA  0.042200
16  jozzH4XCEM9yQFjLAc3_zA  0.036743
6   78hGCrQ6wLA2yh8zMywnmw  0.036141
6   _pBXtjN43eqMV0XZTz7nmw  0.035721
10  5ma1OUQWy_Ds80xUiP7JPQ  0.033874
10  k2b3niokS_tosjah_rzCPw  0.033783
25  LwNZ1AR4_5iukgQYuhmTOg  0.032919
21  I6qFvR-jVDGqDD_i

0   IMLrj2klosTFvPRLv56cng  0.260358
24  ujmEBvifdJM6h6RLv4wQIg  0.050891
68  V13IWgZvNps2Vo7WyEnD8A  0.024049
0   XXW_OFaYQkkGOGniujZFHg  0.000000
0   FvVSy2r7_zDEhZWqLgjXNQ  0.000000
0   WRQ-9LluyivReFiQZFUujw  0.000000
0   Vs7gc9EE3k9wARuUcN9piA  0.000000
0   Os1n1_idfw9vv9kwULGJnQ  0.000000
0   qsJkjQoM3Hdg6Gtqp2S9_w  0.000000
0   O7UMzd3i-Zk8dMeyY9ZwoA  0.000000
0   qmymSqVwHYRqdwfcBatzpQ  0.000000
0   pH0BLkL4cbxKzu471VZnuA  0.000000
0   78hGCrQ6wLA2yh8zMywnmw  0.000000
0   _pBXtjN43eqMV0XZTz7nmw  0.000000
0   tMYYSWyTppDcF070Hmy-kQ  0.000000
0   dm6sO_Y8JdKTE1ZM955yug  0.000000
0   Yiw2jUJzUdFmglHZgj4RaQ  0.000000
0   FhIeCF6QrsLaRvAeu0oEPQ  0.000000
0   5ma1OUQWy_Ds80xUiP7JPQ  0.000000
0   k2b3niokS_tosjah_rzCPw  0.000000